# Introduction

Testing on mountaincar -- haven't updated yet.

1. I changed the keras code so that I could use it to update the actor model with a custom gradient
2. I am adding parameter noise, as suggested by OpenAI

In [ ]:
import gym
import time
import numpy as np
import matplotlib.pyplot as plt
from agent import Agent
%matplotlib inline

#Environment
env = gym.make('MountainCarContinuous-v0')
env.seed(1)  # for comparison
num_states = env.observation_space.shape[0]
num_actions = env.action_space.shape[0]

#Agent
lr,gamma = 0.001, 0.999
agent = Agent(num_states, num_actions, lr, gamma)
agent.memory_size = 10**4
agent.batchsize = 256
learning_start = 25*agent.batchsize
agent.tau = 0.1


#Train
EPISODES = 100
MAX_STEPS = 500
scores = []
for e in range(1,EPISODES+1):
    state = env.reset()
    reward_sum = 0
    done = False
    steps = 0
    t1 = time.time()
    
    while not done:
        #env.render()
        state = np.reshape(state, [1, num_states])  #reshape for keras
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward_sum += reward
        agent.remember(state[0], action, reward, next_state, done)
        state = next_state
        
        if len(agent.memory) > learning_start:
            agent.train_models()
        
        steps += 1
        if done or steps > MAX_STEPS:
            break
    
    #Learn & print results
    scores.append(reward_sum)
    t2 = time.time()
    if e % 1 == 0:
        print '(episode, score, steps, T (mins)) = ' + str((e,reward_sum, steps, (t2-t1)/60.0))

plt.plot(scores)
#np.savetxt('stats/scores_inverted_pendulum.txt',scores)

WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
(episode, score, steps, T (mins)) = (1, -7.148052278566815e-05, 501, 0.006522516409556071)
(episode, score, steps, T (mins)) = (2, -7.029093636288299e-05, 501, 0.004264251391092936)
(episode, score, steps, T (mins)) = (3, -7.015280943222123e-05, 501, 0.004777717590332031)
(episode, score, steps, T (mins)) = (4, -7.015857776918451e-05, 501, 0.004269536336263021)
(episode, score, steps, T (mins)) = (5, -7.128231167851985e-05, 501, 0.004713384310404459)
(episode, score, steps, T (mins)) = (6, -7.043773320240176e-05, 501, 0.0043233672777811686)
(episode, score, steps, T (mins)) = (7, -7.039149443206517e-05, 501, 0.005106767018636067)
(episode, score, steps, T (mins)) = (8, -7.029233571598672e-05, 501, 0.004349752267201742)
(episode, score, steps, T (mins)) = (9, -7.086635839521324e-05, 501, 0.0045

### Watch a smart agent

In [4]:
done = False
while not done:
    env.render()
    state = np.reshape(state, [1, num_states])  #reshape for keras
    action = agent.act(state)
    next_state, reward, done, _ = env.step(action)
    reward_sum += reward
    agent.remember(state[0], action, reward, next_state, done)
    state = next_state
    if done:
        break
env.close()